In [57]:
import xmltodict
import io
import xml.etree.ElementTree as ET
import pandas as pd
import helper
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import numpy as np
import os
#from gensim import corpora, models, similarities 

time: 6 ms


In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime


In [39]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

time: 3 ms


In [15]:
ls data

 Volume in drive E has no label.
 Volume Serial Number is 19C7-A2B3

 Directory of E:\metis\ds4_fletcher\data

08/17/2015  12:55 PM    <DIR>          .
08/17/2015  12:55 PM    <DIR>          ..
03/09/2015  06:04 PM        60,201,826 Badges.xml
03/09/2015  06:04 PM       238,716,194 Comments.xml
03/09/2015  06:05 PM     1,144,804,160 PostHistory.xml
03/09/2015  06:06 PM         6,192,845 PostLinks.xml
03/09/2015  06:06 PM       736,314,717 Posts.xml
03/09/2015  06:06 PM           367,147 Tags.xml
03/09/2015  06:06 PM       100,081,558 Users.xml
03/09/2015  06:07 PM       210,233,404 Votes.xml
               8 File(s)  2,496,911,851 bytes
               2 Dir(s)  638,832,025,600 bytes free
time: 407 ms


In [58]:
data_dir='data'

time: 2 ms


In [59]:
posts_df = helper.get_xml_file_as_df(os.path.join(data_dir,'Posts.xml'),max_num_children=1000)

time: 1min 11s


In [9]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorDisplayName,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,NaN,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,NaN,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654
2,NaN,NaN,<p>You could use a filter on the recording. If...,NaN,0,NaN,2009-07-08T16:34:52.867,NaN,25,2009-07-15T06:48:44.300,...,NaN,NaN,Burkhard,9637,24,2,0,NaN,NaN,NaN
3,NaN,NaN,<p>try out something like:</p>\n\n<p>audible (...,NaN,0,NaN,2009-07-08T16:59:00.587,NaN,26,2009-07-15T06:48:44.300,...,NaN,NaN,Jas Panesar,23242,24,2,0,NaN,NaN,NaN
4,NaN,NaN,<p>You definitely need some sort of software t...,NaN,1,NaN,2009-07-08T17:15:59.477,NaN,27,2009-07-15T06:48:44.300,...,NaN,NaN,T Pops,45675,24,2,0,NaN,NaN,NaN


In [10]:
posts_df.columns

Index([u'AcceptedAnswerId', u'AnswerCount', u'Body', u'ClosedDate',
       u'CommentCount', u'CommunityOwnedDate', u'CreationDate',
       u'FavoriteCount', u'Id', u'LastActivityDate', u'LastEditDate',
       u'LastEditorDisplayName', u'LastEditorUserId', u'OwnerDisplayName',
       u'OwnerUserId', u'ParentId', u'PostTypeId', u'Score', u'Tags', u'Title',
       u'ViewCount'],
      dtype='object')

## Process Text

In [40]:
# strip html tags
posts_df['BodyCleaned']=posts_df['Body'].apply(lambda x : helper.strip_tags(x))
# tokenize text
posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.tokenize_and_stem(x))

# remove stop words
posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.remove_stopwords(x))



time: 4.04 s


In [41]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount,BodyCleaned
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584,"[vista, virtual, machin, use, onli, gb, virtua..."
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654,"[use, record, import, technic, meet, demo, use..."
2,NaN,NaN,<p>You could use a filter on the recording. If...,NaN,0,NaN,2009-07-08T16:34:52.867,NaN,25,2009-07-15T06:48:44.300,...,NaN,Burkhard,9637,24,2,0,NaN,NaN,NaN,"[could, use, filter, record, rememb, well, low..."
3,NaN,NaN,<p>try out something like:</p>\n\n<p>audible (...,NaN,0,NaN,2009-07-08T16:59:00.587,NaN,26,2009-07-15T06:48:44.300,...,NaN,Jas Panesar,23242,24,2,0,NaN,NaN,NaN,"[tri, someth, like, audibl, mac, soundforg, pc..."
4,NaN,NaN,<p>You definitely need some sort of software t...,NaN,1,NaN,2009-07-08T17:15:59.477,NaN,27,2009-07-15T06:48:44.300,...,NaN,T Pops,45675,24,2,0,NaN,NaN,NaN,"[definit, need, sort, softwar, filter, nois, a..."


time: 45 ms


In [42]:
len(posts_df)

1001

time: 3 ms


In [43]:
texts = posts_df['BodyCleaned']

time: 1e+03 µs


In [44]:
#create a Gensim dictionary from the texts
dictionary = gensim.corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

time: 255 ms


In [46]:
corpus

[[(17, 3),
  (35, 1),
  (96, 1),
  (101, 1),
  (108, 3),
  (119, 1),
  (123, 1),
  (165, 1),
  (171, 1),
  (179, 1),
  (190, 1),
  (213, 1),
  (240, 1),
  (263, 1),
  (265, 1),
  (267, 4),
  (292, 1),
  (300, 1),
  (313, 1),
  (344, 5),
  (353, 1),
  (357, 1),
  (358, 1),
  (383, 2),
  (417, 2),
  (453, 1),
  (479, 1),
  (530, 1),
  (557, 1),
  (607, 1),
  (643, 2),
  (655, 9),
  (707, 1),
  (755, 1),
  (787, 1),
  (789, 1),
  (828, 2),
  (880, 1),
  (912, 1),
  (919, 2),
  (925, 2),
  (936, 1),
  (937, 1),
  (1015, 1),
  (1046, 5),
  (1110, 1),
  (1155, 1),
  (1157, 1),
  (1175, 3),
  (1186, 1),
  (1198, 1),
  (1209, 2),
  (1219, 1),
  (1225, 2),
  (1255, 1),
  (1290, 1),
  (1308, 2),
  (1365, 1),
  (1373, 7),
  (1393, 1),
  (1397, 1),
  (1427, 1),
  (1447, 1),
  (1449, 1),
  (1499, 1),
  (1514, 1),
  (1570, 7),
  (1594, 1),
  (1630, 1),
  (1632, 4),
  (1666, 2),
  (1688, 4),
  (1691, 1),
  (1696, 2),
  (1700, 1),
  (1740, 1),
  (1757, 1),
  (1796, 1),
  (1810, 1),
  (1813, 1),
  (181

time: 3.52 s


In [52]:
#gensim.models.LdaMulticore
lda = gensim.models.LdaModel(corpus, num_topics=5, 
                            id2word=dictionary, 
                            update_every=5, 
                            chunksize=10000,
                            passes=100)

time: 10min 11s


In [53]:
lda.show_topics()


[u"0.019*key + 0.010*window + 0.009*user + 0.009*command + 0.009*sudo + 0.008*keyboard + 0.007*n't + 0.007*use + 0.007*click + 0.006*languag",
 u"0.019*use + 0.010*n't + 0.010*'s + 0.008*time + 0.007*mac + 0.007*backup + 0.007*work + 0.006*doe + 0.006*window + 0.006*firefox",
 u"0.016*drive + 0.016*window + 0.015*use + 0.012*instal + 0.012*file + 0.011*n't + 0.008*folder + 0.006*'s + 0.006*one + 0.006*want",
 u"0.030*use + 0.012*window + 0.011*work + 0.009*connect + 0.009*server + 0.008*'s + 0.008*email + 0.007*would + 0.007*user + 0.007*way",
 u"0.014*use + 0.013*'s + 0.012*n't + 0.009*like + 0.008*monitor + 0.008*get + 0.007*window + 0.007*work + 0.007*one + 0.006*fan"]

time: 8 ms


In [56]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

['key', 'window', 'user', 'command', 'sudo', 'keyboard', "n't", 'use', 'click', 'languag', 'run', 'shortcut', 'registri', 'directori', 'su', 'applic', 'disabl', 'set', 'tab', 'ctrl']
()
['use', "n't", "'s", 'time', 'mac', 'backup', 'work', 'doe', 'window', 'firefox', 'ani', 've', 'like', 'also', "'m", 'tri', 'machin', 'disk', 'differ', 'one']
()
['drive', 'window', 'use', 'instal', 'file', "n't", 'folder', "'s", 'one', 'want', 'need', 'run', 'comput', 'flash', 'work', 'ani', 'set', 'itun', 'hard', 'like']
()
['use', 'window', 'work', 'connect', 'server', "'s", 'email', 'would', 'user', 'way', 'like', "n't", 'googl', 'comput', 'also', 'set', 'get', 'ani', 'http', 'need']
()
['use', "'s", "n't", 'like', 'monitor', 'get', 'window', 'work', 'one', 'fan', 've', 'run', 'comput', 'ani', "'m", 'system', 'would', 'card', 'time', 'doe']
()
time: 66 ms
